## 기존 데이터 로드

In [1]:
import pandas as pd

In [2]:
train_file_path = '../aiffel/train.csv'
test_file_path = '../aiffel/test.csv'

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

In [3]:
print('train_df 전체 샘플 수 :', len(train_df))
print('test_df 전체 샘플 수 :', len(test_df))

train_df 전체 샘플 수 : 3950
test_df 전체 샘플 수 : 500


In [4]:
train_df.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [5]:
test_df.head()

,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


## 텍스트 전처리

In [6]:
from glob import glob

In [7]:
# AIHUB에서 수집한 데이터 로드
def load_collected_data(file_path):
    df = pd.read_csv(file_path)
    
    df['class'] = "일반 대화"
    
    # 대화를 몇 번 주고 받았는지 확인
    df["turn_num"] = df["conversation"].apply(lambda x : len(x.split("\n")))
    
    
    # 너무 긴 대화는 삭제
    df = df[(df["turn_num"] >= 4) & (df["turn_num"] <= 15)][["class", "conversation"]]
    print(len(df))
    
    return df

In [8]:
# 결측치 제거
def remove_nan(x):
    # 결측치나 빈 문자열이 있으면 None을 반환 (삭제 효과)
    if pd.isna(x) or x == '':
        return None  # None을 반환하면 해당 값이 삭제됨
    return x

In [27]:
# 특수문자, 공백 제거
def remove_punctuation(x):
  x = re.sub("[^ㄱ-ㅎ가-힣]+", " ", x)
  x = re.sub("[ ]+", " ", x)
  x = x.strip()
  return x

In [9]:
# 라벨 정수 인코딩
def label_encoder(x):
    label_map = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
    x['encoded_label'] = x['class'].map(label_map)
    print(x['encoded_label'].unique())
    
    return x

## 함수 적용

In [10]:
# 수집한 일반 대화 클래스 데이터셋 로드 
file_path_1 = '../aiffel/emotion_dialogue.csv' # AIHUB 감성 대화
file_path_2 = '../aiffel/korean_dialogue.csv' # AIHUB 한국어 대화
file_path_3 = '../aiffel/instagram.csv' # 인스타그램 대화
file_path_4 = '../aiffel/band.csv' # 밴드 대화

normal_train_df_1 = load_collected_data(file_path_1)
normal_train_df_2 = load_collected_data(file_path_2)
normal_train_df_3 = load_collected_data(file_path_3)
normal_train_df_4 = load_collected_data(file_path_4)

58271
4256
570
593


In [28]:
# 훈련 데이터 모두 합치고 확인
train = pd.concat([train_df, normal_train_df_1, normal_train_df_2, normal_train_df_3, normal_train_df_4], axis = 0)
print(train.shape)
print(train.groupby("class").count())
train.head()

(67640, 3)
              idx  conversation
class                          
갈취 대화         981           981
기타 괴롭힘 대화    1094          1094
일반 대화           0         63690
직장 내 괴롭힘 대화   979           979
협박 대화         896           896


,idx,class,conversation
0,0.0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1.0,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2.0,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3.0,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4.0,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [29]:
# 정수 라벨링
label_map = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
train['encoded_label'] = train['class'].map(label_map)
print(train['encoded_label'].unique())

train.head()

[0 3 1 2 4]


,idx,class,conversation,encoded_label
0,0.0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,1.0,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,0
2,2.0,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,3
3,3.0,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,1
4,4.0,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1


In [20]:
from tqdm import tqdm
import re

---
## 구분자 제거 (preprocessed_train.csv)

In [30]:
# 앞서 정의한 함수를 데이터셋에 적용 후 확인
train["conversation"] = train["conversation"].apply(remove_nan)
train["conversation"] = train["conversation"].apply(lambda x : remove_punctuation(x))

train = train.sample(frac=1)
train = train.reset_index(drop=True)
train.drop(columns=['idx'], inplace=True)

print(train.shape)
print(train.groupby("class").count())

train.head()

(67640, 3)
             conversation  encoded_label
class                                   
갈취 대화                 981            981
기타 괴롭힘 대화            1094           1094
일반 대화               63690          63690
직장 내 괴롭힘 대화           979            979
협박 대화                 896            896


,class,conversation,encoded_label
0,일반 대화,모임만 가면 여행을 갔다고 자랑하는 친구 때문에 짜증이 나 친구 때문에 짜증이 나셨...,4
1,일반 대화,직장 동료 중에 명예퇴직 당한 사람이 있어 안타까운 이야기네요 마음이 어떠세요 나도...,4
2,일반 대화,건강 검진을 했는데 수치를 어떻게 해석해야 하는 건지 모르겠어 건강 검진 결과의 수...,4
3,일반 대화,아내가 아이들 결혼자금으로 모아두었던 노후 자금을 쓰자고 해서 당혹스러워 갑자기 아...,4
4,일반 대화,친구가 몸 좋은 나를 부러워해서 자꾸 많이 먹이려고 헤서 화가 나 시기가 많은 친구...,4


In [31]:
# 원 라벨 컬럼은 drop
train.drop(columns=['class'], inplace=True)

train.head()

,conversation,encoded_label
0,모임만 가면 여행을 갔다고 자랑하는 친구 때문에 짜증이 나 친구 때문에 짜증이 나셨...,4
1,직장 동료 중에 명예퇴직 당한 사람이 있어 안타까운 이야기네요 마음이 어떠세요 나도...,4
2,건강 검진을 했는데 수치를 어떻게 해석해야 하는 건지 모르겠어 건강 검진 결과의 수...,4
3,아내가 아이들 결혼자금으로 모아두었던 노후 자금을 쓰자고 해서 당혹스러워 갑자기 아...,4
4,친구가 몸 좋은 나를 부러워해서 자꾸 많이 먹이려고 헤서 화가 나 시기가 많은 친구...,4


In [32]:
# 정수 인코딩 컬럼은 class로 rename
train.rename(columns={'encoded_label': 'class'}, inplace=True)

train.head()

,conversation,class
0,모임만 가면 여행을 갔다고 자랑하는 친구 때문에 짜증이 나 친구 때문에 짜증이 나셨...,4
1,직장 동료 중에 명예퇴직 당한 사람이 있어 안타까운 이야기네요 마음이 어떠세요 나도...,4
2,건강 검진을 했는데 수치를 어떻게 해석해야 하는 건지 모르겠어 건강 검진 결과의 수...,4
3,아내가 아이들 결혼자금으로 모아두었던 노후 자금을 쓰자고 해서 당혹스러워 갑자기 아...,4
4,친구가 몸 좋은 나를 부러워해서 자꾸 많이 먹이려고 헤서 화가 나 시기가 많은 친구...,4


In [33]:
print(train.groupby("class").count())

       conversation
class              
0               896
1               981
2               979
3              1094
4             63690


In [34]:
train.to_csv("../aiffel/preprocessed_train.csv")

---
## 구분자 포함 (preprocessed_train_raw.csv)

In [22]:
train["conversation"] = train["conversation"].apply(remove_nan)

train = train.sample(frac=1)
train = train.reset_index(drop=True)
train.drop(columns=['idx'], inplace=True)

print(train.shape)
print(train.groupby("class").count())

train.head()

(67640, 3)
             conversation  encoded_label
class                                   
갈취 대화                 981            981
기타 괴롭힘 대화            1094           1094
일반 대화               63690          63690
직장 내 괴롭힘 대화           979            979
협박 대화                 896            896


,class,conversation,encoded_label
0,일반 대화,아이가 아빠 생일이라고 그림을 그려줬어.\n좋으시겠어요. 자녀분이 그림을 잘 그리나...,4
1,갈취 대화,야 안녕\n .어 안녕.\n 나 오늘 택시비가 없는데 혹시 빌려줄 수 있어 ??\n...,1
2,일반 대화,친구들이 내 결혼식장에 오면 우리 부부를 보여주는 게 용기가 나질 않아.\n친구들에...,4
3,일반 대화,아들이 이번에 고등학교 졸업했어.\n이번에 졸업을 하셨다니 졸업 축하 드려요.\n근...,4
4,일반 대화,나를 괴롭히던 친구들이 전학을 갔고 안 좋은 사건들을 선생님이 해결해 주어서 기뻐....,4


In [23]:
# 원 라벨 컬럼은 drop
train.drop(columns=['class'], inplace=True)

train.head()

,conversation,encoded_label
0,아이가 아빠 생일이라고 그림을 그려줬어.\n좋으시겠어요. 자녀분이 그림을 잘 그리나...,4
1,야 안녕\n .어 안녕.\n 나 오늘 택시비가 없는데 혹시 빌려줄 수 있어 ??\n...,1
2,친구들이 내 결혼식장에 오면 우리 부부를 보여주는 게 용기가 나질 않아.\n친구들에...,4
3,아들이 이번에 고등학교 졸업했어.\n이번에 졸업을 하셨다니 졸업 축하 드려요.\n근...,4
4,나를 괴롭히던 친구들이 전학을 갔고 안 좋은 사건들을 선생님이 해결해 주어서 기뻐....,4


In [24]:
# 정수 인코딩 컬럼은 class로 rename
train.rename(columns={'encoded_label': 'class'}, inplace=True)

train.head()

,conversation,class
0,아이가 아빠 생일이라고 그림을 그려줬어.\n좋으시겠어요. 자녀분이 그림을 잘 그리나...,4
1,야 안녕\n .어 안녕.\n 나 오늘 택시비가 없는데 혹시 빌려줄 수 있어 ??\n...,1
2,친구들이 내 결혼식장에 오면 우리 부부를 보여주는 게 용기가 나질 않아.\n친구들에...,4
3,아들이 이번에 고등학교 졸업했어.\n이번에 졸업을 하셨다니 졸업 축하 드려요.\n근...,4
4,나를 괴롭히던 친구들이 전학을 갔고 안 좋은 사건들을 선생님이 해결해 주어서 기뻐....,4


In [25]:
print(train.groupby("class").count())

       conversation
class              
0               896
1               981
2               979
3              1094
4             63690


In [26]:
train.to_csv("../aiffel/preprocessed_train_raw.csv")